# Level Net 2


## Python 最小二乘法模組

In [1]:
import numpy as np
from numpy.linalg import inv
import math

# General purpose matrix printing function
def Matrix_print(X,matrix_title="Matrix"):
    print("\n%s"%matrix_title)
    
    if len(X.shape)==1:
        for x in X:
            print('%10.4f'%(x))
    else:
        print(X)
        
# General purpose matrix printing function
def Matrix_print2(X,sX,matrix_title="Matrix",unit=""):
    print("\n%s"%matrix_title)
    
    for x,sx in zip(X,sX):
        if unit is None:
            print('%.4f +/- %.4f'%(x,sx))
        else:
            print('%.4f%s +/- %.4f (%s)'%(x, unit, sx, unit))

In [2]:
import numpy as np
from numpy.linalg import inv
import math

def LESA(A,L, p=None, px=None, unit=None):
    m,n=A.shape
    #print(m,n)

    #% 若未賦予權值，以等權計算
    if p is None:
        P=np.diag(np.ones(m),0)
    else:
        P=np.diag(p)  

    #% 未知數的答解 
    #% 法方程式之反矩陣
    # N=A'*P*A;
    N=np.dot(A.T,P)
    N=np.dot(N,A)

    #% 如果有提供未知數加權參數
    if px is not None:
        px=diag(px)
        N=N+px

    # %N_inv=inv(N);
    # U=A'*P*L;
    U=np.dot(A.T,P)
    U=np.dot(U,L)

    N_inv=inv(N)
    X=np.dot(N_inv,U)

    #% 誤差分析
    #  V=A*X-L;
    V=np.dot(A,X)-L

    #%單位權標準誤差   
    #sigma0=sqrt((V'*P*V)./(m-n));
    q=np.dot(V.T,P)
    q=np.dot(q,V)
    sigma0=math.sqrt(q/(m-n))

    #%未知數精度矩陣
    DX=N_inv*(sigma0**2)
    sX=np.diag(DX)**0.5

    #%觀測量精度分析   
    DL=np.dot(A,N_inv)
    DL=sigma0**2*np.dot(DL,A.T)
    sL=(np.diag(DL))**0.5

    return X, V, sigma0, DX, sL, P, N, U

In [3]:
def LESA_Print(A,L, X, V, sigma0, DX, sL, P, N, U):
    print('***** 最小二乘法計算模組報表 *****')
    
    Matrix_print(A,'設計矩陣A')
    Matrix_print(L,'觀測矩陣L')
    
    Matrix_print(P,'加權矩陣P')
    Matrix_print(N,'法方程式 N矩陣')
    Matrix_print(U,'法方程式 U矩陣')
    Matrix_print(X,'未知數X')
    Matrix_print(V,'誤差矩陣V')

    print('\n單位權標準誤差：+/- %.4f'%(sigma0) )

    sX=np.diag(DX)**0.5
    Matrix_print2(X,sX,'未知數及精度分析',unit='m')
    Matrix_print(sL,'觀測量精度矩陣')

# Wolf 課本12.3 未加權範例


In [4]:
A=np.array([
   [1,   0,   0],
   [-1,  0,   0],
   [0,   0,   1],
   [0,   0,  -1],
   [-1,  1,   0],
   [0,   1,   0],
   [0,  -1,   1]
])

B=np.array([
   -100.00,
    107.50,
   -107.50,
    100.00,
      0.00,
   -107.50,
      0.00
])

L=np.array([
      5.10,
      2.34,
     -1.25,
     -6.13,
     -0.68,
     -3.00,
      1.70
])
L=L-B
L=L.T

In [5]:
# 無加權矩陣
X, V, sigma0, DX, sL, P, N, U=LESA(A,L,unit='m')
LESA_Print(A,L, X, V, sigma0, DX, sL, P, N, U)

***** 最小二乘法計算模組報表 *****

設計矩陣A
[[ 1  0  0]
 [-1  0  0]
 [ 0  0  1]
 [ 0  0 -1]
 [-1  1  0]
 [ 0  1  0]
 [ 0 -1  1]]

觀測矩陣L
  105.1000
 -105.1600
  106.2500
 -106.1300
   -0.6800
  104.5000
    1.7000

加權矩陣P
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]

法方程式 N矩陣
[[ 3. -1.  0.]
 [-1.  3. -1.]
 [ 0. -1.  3.]]

法方程式 U矩陣
  210.9400
  102.1200
  214.0800

未知數X
  105.1410
  104.4829
  106.1876

誤差矩陣V
    0.0410
    0.0190
   -0.0624
   -0.0576
    0.0219
   -0.0171
    0.0048

單位權標準誤差：+/- 0.0501

未知數及精度分析
105.1410m +/- 0.0309 (m)
104.4829m +/- 0.0328 (m)
106.1876m +/- 0.0309 (m)

觀測量精度矩陣
    0.0309
    0.0309
    0.0309
    0.0309
    0.0363
    0.0328
    0.0363


# Wolf 課本12.4 加權範例

In [6]:
# 指定加權矩陣的使用方法
w=np.array([3,4,6,4,6,6,6])
X, V, sigma0, DX, sL, P, N, U=LESA(A,L,p=w,unit='m')
LESA_Print(A,L, X, V, sigma0, DX, sL, P, N, U)

***** 最小二乘法計算模組報表 *****

設計矩陣A
[[ 1  0  0]
 [-1  0  0]
 [ 0  0  1]
 [ 0  0 -1]
 [-1  1  0]
 [ 0  1  0]
 [ 0 -1  1]]

觀測矩陣L
  105.1000
 -105.1600
  106.2500
 -106.1300
   -0.6800
  104.5000
    1.7000

加權矩陣P
[[3 0 0 0 0 0 0]
 [0 4 0 0 0 0 0]
 [0 0 6 0 0 0 0]
 [0 0 0 4 0 0 0]
 [0 0 0 0 6 0 0]
 [0 0 0 0 0 6 0]
 [0 0 0 0 0 0 6]]

法方程式 N矩陣
[[13 -6  0]
 [-6 18 -6]
 [ 0 -6 16]]

法方程式 U矩陣
  740.0200
  612.7200
 1072.2200

未知數X
  105.1504
  104.4892
  106.1972

誤差矩陣V
    0.0504
    0.0096
   -0.0528
   -0.0672
    0.0188
   -0.0108
    0.0080

單位權標準誤差：+/- 0.1072

未知數及精度分析
105.1504m +/- 0.0328 (m)
104.4892m +/- 0.0298 (m)
106.1972m +/- 0.0290 (m)

觀測量精度矩陣
    0.0328
    0.0328
    0.0290
    0.0290
    0.0338
    0.0298
    0.0326


# Wold課本12.6範例

In [7]:
A=np.array([
   [1,   0,   0],
   [-1,  1,   0],
   [0,  -1,   1],
   [0,   0,  -1],
   [-1,  0,   1],
   [0,   1,   0]
])

L=np.array([
    448.105,
      5.360,
     -8.523,
   -444.944,
     -3.167,
    453.477
])

L=L.T

In [8]:
# 加權矩陣
s=np.array([0.006,0.004,0.005,0.003,0.004,0.012])
w=1/(s*s)
X, V, sigma0, DX, sL, P, N, U=LESA(A,L,p=w,unit='m')
LESA_Print(A,L, X, V, sigma0, DX, sL, P, N, U)

***** 最小二乘法計算模組報表 *****

設計矩陣A
[[ 1  0  0]
 [-1  1  0]
 [ 0 -1  1]
 [ 0  0 -1]
 [-1  0  1]
 [ 0  1  0]]

觀測矩陣L
  448.1050
    5.3600
   -8.5230
 -444.9440
   -3.1670
  453.4770

加權矩陣P
[[ 27777.77777778      0.              0.              0.
       0.              0.        ]
 [     0.          62500.              0.              0.
       0.              0.        ]
 [     0.              0.          40000.              0.
       0.              0.        ]
 [     0.              0.              0.         111111.11111111
       0.              0.        ]
 [     0.              0.              0.              0.
   62500.              0.        ]
 [     0.              0.              0.              0.
       0.           6944.44444444]]

法方程式 N矩陣
[[152777.77777778 -62500.         -62500.        ]
 [-62500.         109444.44444444 -40000.        ]
 [-62500.         -40000.         213611.11111111]]

法方程式 U矩陣
12310298.6111
3825065.8333
48899364.7222

未知數X
  448.1087
  453.4685
  444.